In [2]:
import pandas as pd
from os.path import join
import os
import numpy as np
import matplotlib.pyplot as plt
from math import ceil

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import cv2

from torch_predict import predict
torch.manual_seed(42)

In [5]:
project_dir = os.getcwd()
data_dir = join(project_dir, "dog-breeds-data", "images", "Images")
csv_file = join(project_dir, "dog-breeds-data", "index.csv")
tsinghua_data_dir = join(project_dir, "dog-breeds-data-2", "classes")
tsinghua_csv_file = join(project_dir, "dog-breeds-data-2", "index.csv")

df1, df2 = pd.read_csv(csv_file), pd.read_csv(tsinghua_csv_file)
df = pd.concat([df1, df2])
df = df.sample(frac=1)

print(len(df1), len(df2), len(df))

df.head()

20580 70432 91012


,breed,imagePath
9425,german_short-haired_pointer,dog-breeds-data/images/Images/n02100236-German...
63543,golden_retriever,dog-breeds-data-2/classes/5355-n000126-golden_...
17269,appenzeller,dog-breeds-data/images/Images/n02107908-Appenz...
39036,samoyed,dog-breeds-data-2/classes/2192-n000088-Samoyed...
44574,chinese_rural_dog,dog-breeds-data-2/classes/3336-n000121-chinese...


In [110]:
# df["imagePath"].replace("Shih-Tzu", "Shih_Tzu", inplace=True)
# df.to_csv(csv_file, index=False)

In [104]:

# temp_df = pd.read_csv(tsinghua_csv_file)
# temp_df["imagePath"] = 'dog-breeds-data-2/classes/' + temp_df["imagePath"].astype(str)
# temp_df.head()

,breed,imagePath
0,fila_braziliero,dog-breeds-data-2/classes/220-n000032-Fila_Bra...
1,fila_braziliero,dog-breeds-data-2/classes/220-n000032-Fila_Bra...
2,fila_braziliero,dog-breeds-data-2/classes/220-n000032-Fila_Bra...
3,fila_braziliero,dog-breeds-data-2/classes/220-n000032-Fila_Bra...
4,fila_braziliero,dog-breeds-data-2/classes/220-n000032-Fila_Bra...


In [105]:
# temp_df.to_csv(tsinghua_csv_file, index=False)

In [118]:
incorrect_preds = [] # prediction, actual
paths = []

for i, row in df.iterrows():
    prediction = predict(join(project_dir, row["imagePath"]))[0][0]
    if prediction != row["breed"]:
        incorrect_preds.append((prediction, row["breed"]))
        paths.append(row["imagePath"])
    if len(incorrect_preds) >= 30:
        break
    print(paths, end='\r')



In [98]:
df = pd.read_csv(csv_file)
stanford_classes = sorted(df["breed"].unique())

tsinghua_df = pd.read_csv(tsinghua_csv_file)
tsinghua_classes = sorted(tsinghua_df["breed"].unique())
# tsinghua_classes = sorted([folder.split('-', 2)[-1].lower() for folder in os.listdir(tsinghua_data_dir) 
#                            if os.path.isdir(join(tsinghua_data_dir, folder))])

stanfordSet, tsinghuaSet = set(stanford_classes), set(tsinghua_classes)

print("# Stanford dataset classes:", len(stanford_classes))
print("# Tsinghua dataset classes:", len(tsinghua_classes))
print("# Common classes:", len(stanfordSet.intersection(tsinghuaSet)))

print("\nStanford classes not in Tsinghua classes:", sorted(list(stanfordSet - tsinghuaSet)))
print("\nTsinghua classes not in Stanford classes:", sorted(list(tsinghuaSet - stanfordSet)))

# Stanford dataset classes: 120
# Tsinghua dataset classes: 130
# Common classes: 120

Stanford classes not in Tsinghua classes: []

Tsinghua classes not in Stanford classes: ['australian_shepherd', 'bichon_frise', 'black_sable', 'cane_carso', 'chinese_crested_dog', 'chinese_rural_dog', 'fila_braziliero', 'japanese_spitzes', 'shiba_dog', 'teddy']


In [108]:
for folder in os.listdir(data_dir):
    if "Shi" in folder:
        print(folder)

n02086240-Shih_Tzu


In [91]:
tsinghua_classes = sorted([folder.split('-', 2)[-1].lower() for folder in os.listdir(tsinghua_data_dir) 
                           if os.path.isdir(join(tsinghua_data_dir, folder))])
print(tsinghua_classes)

225-n000062-tan_coonhound
['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale', 'american_staffordshire_terrier', 'appenzeller', 'australian_shepherd', 'australian_terrier', 'basenji', 'basset', 'beagle', 'bedlington_terrier', 'bernese_mountain_dog', 'bichon_frise', 'black_sable', 'blenheim_spaniel', 'bloodhound', 'bluetick', 'border_collie', 'border_terrier', 'borzoi', 'boston_bull', 'bouvier_des_flandres', 'boxer', 'brabancon_griffon', 'briard', 'brittany_spaniel', 'bull_mastiff', 'cairn', 'cane_carso', 'cardigan', 'chesapeake_bay_retriever', 'chihuahua', 'chinese_crested_dog', 'chinese_rural_dog', 'chow', 'clumber', 'coated_wheaten_terrier', 'cocker_spaniel', 'collie', 'curly_coated_retriever', 'dandie_dinmont', 'dhole', 'dingo', 'doberman', 'english_foxhound', 'english_setter', 'english_springer', 'entlebucher', 'eskimo_dog', 'fila_braziliero', 'flat_coated_retriever', 'french_bulldog', 'german_shepherd', 'german_short-haired_pointer', 'giant_schnauzer', 'golden_retr

In [111]:
# # create index csv for stanford dataset
# new_df = pd.DataFrame(columns=["breed", "imagePath"])

# for folder in os.listdir(data_dir):
#     if not os.path.isdir(join(data_dir, folder)): continue

#     for fn in os.listdir(join(data_dir, folder)):
#         if fn[0] == '.': continue

#         temp_df = {"breed": folder.split('-', 1)[-1].lower(), "imagePath": join("dog-breeds-data", "images", "Images", folder, fn)}
#         new_df = new_df.append(temp_df, ignore_index=True)
#     print(sorted(new_df["breed"].unique()), end='\r')

# new_df.to_csv(join(project_dir, "index.csv"), index=False)



In [6]:
# create index csv for tsinghua dataset
new_df = pd.DataFrame(columns=["breed", "imagePath"])

for folder in os.listdir(tsinghua_data_dir):
    if not os.path.isdir(join(tsinghua_data_dir, folder)): continue

    for fn in os.listdir(join(tsinghua_data_dir, folder)):
        if fn[0] == '.': continue

        temp_df = {"breed": folder.split('-', 2)[-1].lower(), "imagePath": join("dog-breeds-data-2", "classes", folder, fn)}
        new_df = new_df.append(temp_df, ignore_index=True)
    print(sorted(new_df["breed"].unique()), end='\r')

new_df.to_csv(join(project_dir, "index.csv"), index=False)

In [77]:
a = sorted(new_df["breed"].unique())
b = [x for x in a if '-' in x]
b

['german_short-haired_pointer']